<a href="https://colab.research.google.com/github/ikedachin/llm_lecture_2024/blob/main/inference_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install libraries

In [4]:
!pip uninstall unsloth -y
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Found existing installation: unsloth 2024.12.9
Uninstalling unsloth-2024.12.9:
  Successfully uninstalled unsloth-2024.12.9
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-lehxe10_/unsloth_edd817a14d40416189ed2a8aad4baca3
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-lehxe10_/unsloth_edd817a14d40416189ed2a8aad4baca3
  Resolved https://github.com/unslothai/unsloth.git to commit a2407835534747d2421f58cbdeeb5a49482e7235
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.12.9-py3-none-any.whl size=174499 sha256=0b0794c4374ed587675b9633cea2cb45d6be4d88f9b03b9fe82604b8e46ccc77
  Stored in directory: /tmp/pip-ephem-wheel-cache-o_hwmk6s/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth


In [2]:
!pip install --upgrade torch
!pip install --upgrade xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 94.5 MB/s eta 0:00:00


In [3]:
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Using cached ninja-1.11.1.3-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (422 kB)
  Created wheel for flash-attn: filename=flash_attn-2.7.2.post1-cp310-cp310-linux_x86_64.whl size=190160474 sha256=0b454d9e650bfc437cc71335080172a5d05f51eab355636c9d5b7321fec7318e
  Stored in directory: /root/.cache/pip/wheels/da/ec/5b/b2c37a8e4f755ad82492a822463bca0817f0e0e11de874b550
Successfully built flash-attn


# import libraries

In [5]:
import re
import json
from tqdm import tqdm
import torch
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Your Flash Attention 2 installation seems to be broken?
A possible explanation is you have a new CUDA version which isn't
yet compatible with FA2? Please file a ticket to Unsloth or FA2.
We shall now use Xformers instead, which does not have any performance hits!
We found this negligible impact by benchmarking on 1x A100.
🦥 Unsloth Zoo will now patch everything to make training faster!


# config

In [6]:
max_seq_length = 1024
dtype = None
load_in_4bit = True
model_id = "ikedachin/llm-jp-3-13b-october-news-e1-all-3-5-sft-llmjp-magpie-20000"

In [7]:
# set Token of Huggingface
from google.colab import userdata
HF_W=userdata.get('HF_TOKEN')

# download model

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    trust_remote_code = True,
)

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2024.12.9: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.9. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

# prepare a dataset for inference

In [ ]:
datasets = []
with open("./elyza-tasks-100-TV_0.jsonl", "r") as f:
    item = ""
    for line in f:
        line = line.strip()
        item += line
        if item.endswith("}"):
            datasets.append(json.loads(item))
            item = ""

# inference

In [ ]:
# 推論するためにモデルのモードを変更
FastLanguageModel.for_inference(model)

results = []
for dt in tqdm(datasets):
    input = dt["input"]

    prompt = f"""### 指示
{input}
### 回答
"""

    inputs = tokenizer([prompt], return_tensors = "pt").to(model.device)

    outputs = model.generate(**inputs, max_new_tokens = max_seq_length, use_cache = True, do_sample=False, repetition_penalty=1.2)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).split('\n### 回答')[-1]

    results.append({"task_id": dt["task_id"], "input": input, "output": prediction})

# save results

In [ ]:
json_file_id = re.sub(".*/", "", model_id)
with open(f"/content/{json_file_id}_output.jsonl", 'w', encoding='utf-8') as f:
    for result in results:
        json.dump(result, f, ensure_ascii=False)
        f.write('\n')